In [18]:
from tensorflow import keras as K
from tensorflow.keras import applications

In [66]:
# import libraries
from tensorflow.keras.applications.vgg16 import VGG16, decode_predictions, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image # Keras own inbuild image class
from tensorflow.keras.layers import Dense,Dropout,GlobalAveragePooling2D,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential,Input,Model

import numpy as np
import matplotlib.pyplot as plt

from PIL import ImageEnhance, ImageFilter
import random
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [20]:
# Function to resize images
def resize_images(folder_path, target_size=(416, 416)):
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        if img_name.endswith(('.png', '.jpg', '.jpeg')):  # Check for valid image files
            img = Image.open(img_path)
            img_resized = img.resize(target_size)
            img_resized.save(img_path)  # Overwrite the original image with the resized version

# Paths to your train, test, and valid folders
train_folder = 'data/train/Healthy'
test_folder = 'data/test/Healthy'
valid_folder = 'data/valid/Healthy'

# Resize images in all folders
resize_images(train_folder)
resize_images(test_folder)
resize_images(valid_folder)


In [21]:
# folder names containing images of the things you want to classify
underrep_classes = ['Manganese Deficiency','PH Fluctuation','Phosphorus Deficiency','Potassium Deficiency','Sulfur Deficiency']
new_class = ['Healthy']

In [22]:
all_classes = underrep_classes + new_class

In [23]:
# define an image data generator
# Data augmentation: Applies random distortions and transformations to the images (only on your training data!).

data_gen = image.ImageDataGenerator(
    # define the preprocessing function that should be applied to all images
    preprocessing_function=preprocess_input,
    fill_mode='nearest',
    rotation_range=360,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # horizontal_flip=True, 
    # zoom_range=0.2,
    # shear_range=0.2
)

In [24]:
# Custom preprocessing function to handle exposure, blur, and noise
def custom_preprocessing_function(img):
    # Convert to PIL Image for easy manipulation
    img = image.array_to_img(img)
    
    # Apply random exposure change (-10% to +10%)
    exposure_factor = random.uniform(0.9, 1.1)  # Between -10% and +10%
    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(exposure_factor)
    
    # Apply blur (up to 1.5px)
    blur_radius = random.uniform(0, 1.5)  # Random blur between 0 and 1.5 pixels
    img = img.filter(ImageFilter.GaussianBlur(blur_radius))
    
    # Add random noise (up to 5%)
    noise_factor = random.uniform(0, 0.05)  # Up to 5% noise
    noise = np.random.normal(loc=0, scale=1, size=(img.size[1], img.size[0], 3)) * noise_factor
    img_array = np.asarray(img) / 255.0  # Normalize to [0,1]
    img_array = img_array + noise  # Add noise
    img_array = np.clip(img_array, 0, 1) * 255.0  # Clip and scale back to [0,255]
    
    # Convert back to array and return
    return image.img_to_array(img_array)

In [47]:
# Data augmentation generator (only for Healthy folder)

data_gen_health = image.ImageDataGenerator(
    # define the preprocessing function that should be applied to all images
    preprocessing_function=custom_preprocessing_function,
    fill_mode='nearest',
    rotation_range=360,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # horizontal_flip=True, 
    # zoom_range=0.2,
    # shear_range=0.2
)

In [63]:
# a generator that returns batches of X and y arrays
train_data_gen = data_gen.flow_from_directory(
    directory='data/train',
    class_mode="categorical",
    classes=underrep_classes,
    batch_size=5,
    target_size=(416, 416),
    save_to_dir='data/train/',
    save_format='jpg',
    save_prefix='aug'
)


Found 240 images belonging to 5 classes.


In [49]:
healthy_data_gen = data_gen_health.flow_from_directory(
    directory='data/train',
    class_mode="categorical",
    classes=new_class,
    batch_size=5,
    target_size=(416, 416),
    save_to_dir='data/train/Healthy',
    save_format='jpg',
    save_prefix='aug'
)

Found 10 images belonging to 1 classes.


In [42]:
print(all_classes)

['Manganese Deficiency', 'PH Fluctuation', 'Phosphorus Deficiency', 'Potassium Deficiency', 'Sulfur Deficiency', 'Healthy']


In [50]:
val_data_gen = data_gen.flow_from_directory(
    directory='data/valid',
    class_mode="categorical",
    classes=all_classes,
    batch_size=5,
    target_size=(416, 416),
)

Found 13 images belonging to 6 classes.


In [44]:
train_data_gen.class_indices

{'Manganese Deficiency': 0,
 'PH Fluctuation': 1,
 'Phosphorus Deficiency': 2,
 'Potassium Deficiency': 3,
 'Sulfur Deficiency': 4}

In [53]:
for i in range(18):
    x_batch, y_batch = healthy_data_gen.__next__()
    print(f"Saved augmented image {i+1}")

Saved augmented image 1
Saved augmented image 2
Saved augmented image 3
Saved augmented image 4
Saved augmented image 5
Saved augmented image 6
Saved augmented image 7
Saved augmented image 8
Saved augmented image 9
Saved augmented image 10
Saved augmented image 11
Saved augmented image 12
Saved augmented image 13
Saved augmented image 14
Saved augmented image 15
Saved augmented image 16
Saved augmented image 17
Saved augmented image 18


In [65]:
for i in range(40):
    x_batch1, y_batch1 = train_data_gen.__next__()
    print(f"Saved augmented image {i+1}")

Saved augmented image 1
Saved augmented image 2
Saved augmented image 3
Saved augmented image 4
Saved augmented image 5
Saved augmented image 6
Saved augmented image 7
Saved augmented image 8
Saved augmented image 9
Saved augmented image 10
Saved augmented image 11
Saved augmented image 12
Saved augmented image 13
Saved augmented image 14
Saved augmented image 15
Saved augmented image 16
Saved augmented image 17
Saved augmented image 18
Saved augmented image 19
Saved augmented image 20
Saved augmented image 21
Saved augmented image 22
Saved augmented image 23
Saved augmented image 24
Saved augmented image 25
Saved augmented image 26
Saved augmented image 27
Saved augmented image 28
Saved augmented image 29
Saved augmented image 30
Saved augmented image 31
Saved augmented image 32
Saved augmented image 33
Saved augmented image 34
Saved augmented image 35
Saved augmented image 36
Saved augmented image 37
Saved augmented image 38
Saved augmented image 39
Saved augmented image 40
